In [23]:
import numpy as np
from collections import defaultdict

In [47]:
import numpy as np
from collections import defaultdict
from tqdm import tqdm

def initialize_partition(graph):
    return {node: node for node in graph}

def calculate_modularity(graph, partition):
    m = sum(len(neighbors) for neighbors in graph.values()) / 2
    q = 0
    for i in graph:
        for j in graph[i]:
            if partition[i] == partition[j]:
                q += 1 - (len(graph[i]) * len(graph[j])) / (2 * m)
    return q / (2 * m)

def calculate_modularity_gain(node, community, graph, partition, m):
    k_i = len(graph[node])
    k_i_in = sum(1 for neighbor in graph[node] if partition[neighbor] == community)
    sigma_tot = sum(len(graph[n]) for n in graph if partition[n] == community)
    return (k_i_in - sigma_tot * k_i / (2 * m))

def find_best_community(node, graph, partition, m):
    current_community = partition[node]
    best_community = current_community
    best_gain = 0
    for neighbor in graph[node]:
        neighbor_community = partition[neighbor]
        if neighbor_community != current_community:
            gain = calculate_modularity_gain(node, neighbor_community, graph, partition, m)
            if gain > best_gain:
                best_gain = gain
                best_community = neighbor_community
    return best_community

def assign_smallest_node_id(partition):
    community_to_nodes = defaultdict(list)
    for node, community in partition.items():
        community_to_nodes[community].append(node)
    
    new_partition = {}
    for community, nodes in community_to_nodes.items():
        new_community_id = min(nodes)
        for node in nodes:
            new_partition[node] = new_community_id
    
    return new_partition

def louvain_phase_one(graph):
    partition = initialize_partition(graph)
    m = sum(len(neighbors) for neighbors in graph.values()) / 2
    improvement = True
    
    while improvement:
        improvement = False
        for node in tqdm(graph):
            best_community = find_best_community(node, graph, partition, m)
            if best_community != partition[node]:
                old_partition = partition.copy()
                partition[node] = best_community
                if calculate_modularity(graph, partition) > calculate_modularity(graph, old_partition):
                    improvement = True
                else:
                    partition = old_partition
    
    return assign_smallest_node_id(partition)

def coalesce_graph(graph, partition):
    new_graph = defaultdict(lambda: defaultdict(int))
    node_to_community = defaultdict(list)
    
    for node, community in partition.items():
        node_to_community[community].append(node)
    
    for node, neighbors in graph.items():
        comm_i = partition[node]
        for neighbor in neighbors:
            comm_j = partition[neighbor]
            new_graph[comm_i][comm_j] += 1
    
    return dict(new_graph), node_to_community

def run_louvain_iteration(graph):
    with tqdm(total=4, desc="Louvain Algorithm Progress") as pbar:
        pbar.set_description("First Phase")
        partition = louvain_phase_one(graph)
        pbar.update(1)

        pbar.set_description("Coalescing Graph")
        coalesced_graph, node_mapping = coalesce_graph(graph, partition)
        pbar.update(1)

        pbar.set_description("Second Phase")
        coalesced_partition = louvain_phase_one(coalesced_graph)
        pbar.update(1)

        pbar.set_description("Finalizing Partition")
        final_partition = {}
        for coalesced_node, new_community in coalesced_partition.items():
            for original_node in node_mapping[coalesced_node]:
                final_partition[original_node] = new_community
        
        n = len(graph)
        graph_partition = np.zeros(n, dtype=int)
        for node, community in final_partition.items():
            graph_partition[node] = community
        pbar.update(1)

    return graph_partition

# Example usage:
if __name__ == "__main__":
    example_graph = {
        0: [1, 2],
        1: [0, 2, 3, 4],
        2: [0, 1, 3, 4],
        3: [1, 2, 4],
        4: [1, 2, 3, 5, 6],
        5: [4, 6],
        6: [4, 5, 7, 8, 9],
        7: [6, 8, 9],
        8: [6, 7, 9, 10],
        9: [6, 7, 8, 10],
        10: [8, 9]
    }

    result = run_louvain_iteration(example_graph)
    print("Resulting graph partition:", result)

Finalizing Partition: 100%|██████████| 4/4 [00:00<00:00, 640.08it/s]

Resulting graph partition: [0 0 0 0 0 0 6 6 6 6 6]


In [53]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

def visualise_dendogram(community_mat):
    Z = linkage(community_mat, method='single', metric='hamming')
    
    plt.figure(figsize=(10, 7))
    dendrogram(Z, labels=np.arange(community_mat.shape[0]))
    
    plt.title("Dendrogram of Louvain Communities")
    plt.xlabel("Node ID")
    plt.ylabel("Distance")
    plt.savefig("dendrogram.png")
    plt.show()

In [29]:
path = './data/wiki-Vote.txt.gz'
import gzip
edges = []
with gzip.open(path, 'rt', encoding='utf-8') as file:
    for line in file:
        if line.startswith('#'):
            continue
        node1, node2 = map(int, line.strip().split())
        edges.append((node1, node2))  
edges[:5], len(edges) 

([(30, 1412), (30, 3352), (30, 5254), (30, 5543), (30, 7478)], 103689)

In [30]:
def map_node_ids(graph):
    unique_ids = sorted(graph.keys())
    id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_ids)}
    new_graph = {}
    for old_id, neighbors in graph.items():
        new_id = id_mapping[old_id]
        new_graph[new_id] = [id_mapping[neighbor] for neighbor in neighbors]
    
    return new_graph, id_mapping

def revert_node_ids(new_graph, original_id_mapping):
    
    reverse_mapping = {new_id: old_id for old_id, new_id in original_id_mapping.items()}
    original_graph = {}
    
    for new_id, neighbors in new_graph.items():
        old_id = reverse_mapping[new_id]
        original_graph[old_id] = [reverse_mapping[neighbor] for neighbor in neighbors]
    
    return original_graph

In [31]:
uniq_nodes = set()
for a,b in edges:
    uniq_nodes.add(a)
    uniq_nodes.add(b)
graph = {node: [] for node in uniq_nodes}
for a, b in edges:
    graph[a].append(b)
    graph[b].append(a)

len(graph)

7115

In [32]:
new_g, id_map = map_node_ids(graph)

In [48]:
res = run_louvain_iteration(new_g)

Finalizing Partition: 100%|██████████| 4/4 [14:13<00:00, 213.46s/it]


In [54]:
res

array([ 0,  0,  0, ..., 12, 12, 12])

In [55]:
np.unique(res)

array([   0,   12,   50, 2115, 2921, 2970, 3760, 4076, 4805, 5033, 5103,
       5271, 5298, 5375, 5384, 5509, 5516, 6132, 6268, 6272, 6472, 6497,
       6857, 6865, 6888, 6935])